<a href="https://colab.research.google.com/github/amitaofficial/Tech_Projects/blob/main/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Read Me:
# 1. Training data should be in a text file named 'train.txt'and test data in 'test.txt'.
# 2. I have attached train.txt (first 90 data points) and test.txt(last 30 data points) which i got for my student id
# 3. maximum depth of the tree can be changed at line number 4, by changing the value to variable 'given_depth'
# Author - Amita Ghosh (1001841234)
# References: # https://stackoverflow.com/questions/903853/how-do-you-extract-a-column-from-a-multi-dimensional-array
              # https://towardsdatascience.com/decision-tree-algorithm-in-python-from-scratch-8c43f0e40173
              # S. Russell and P. Norvig, "Artificial Intelligence: A Modern Approach"


In [ ]:
import math
import numpy as np

given_depth = 3 #mention the maximum depth here

In [ ]:
#function to calculate entropy of a given dataset
def calculate_entropy(data_array):
  data_count = len(data_array)
  if(data_count == 0):#checking if data is present or not
    return 1
  M_count = 0
  W_count = 0
  for data in data_array:
    if(data[3] == 'M'):
      M_count += 1 #count the number of data with 'M' class
    else:
      W_count += 1 #count the number of data with 'W' class

  # print('M:', M_count,'/', data_count)
  # print('W:',W_count,'/', data_count)

  p1 = M_count / data_count # calculate probability of 'M'
  p2 = W_count / data_count # calculate probability of 'W'
  if(p1 > 0 and p2 > 0): # check if probabilities are greater than 0 to avoid 0 * 0 multiplication
    H = -(p1 * math.log2(p1)) - (p2 * math.log2(p2))  
    return H
  elif (p1 > 0):
    return -(p1 * math.log2(p1))
  elif (p2 > 0):
    return -(p2 * math.log2(p2))
  else:
    return 0

In [ ]:
#function to choose the right attribute-threshold combo for the next node
def choose_attribute(data_array,no_of_attributes):
  best_threshold = 0.0 # best threshold that gives highest info.gain
  best_attribute = 0 # best attribute that gives highest info.gain
  best_gain = 0.0 #best gain out of all attribute-threshold combinations
  for k in range(no_of_attributes):
    #sort feature
    f = [row[k] for row in data_array] #get the column feature
    f.sort()
    #calculating thresholds
    f_thresholds = []
    for i in range(len(f)-1):
      threshold = (f[i]+f[i+1])/2
      f_thresholds.append(threshold)
    # print('Attribute No : ',k+1)
    
    #calculate before split entropy
    H_initial = calculate_entropy(data_array)
    # print('H_initial:',H_initial)
    
    #calculate entropy after split for each threshold
    data_before_split = data_array
    selected_threshold = 0.0
    max_gain = 0.0
    for threshold in f_thresholds:
      # print('threshold is : ',threshold)
      data_after_split_left = []
      data_after_split_right = []
      for data in data_array:
        if(data[k] <= threshold):
          data_after_split_left.append(data)
        else:
          data_after_split_right.append(data)
      #calculate entropy after split
      # print('left split :')
      H1 = calculate_entropy(data_after_split_left)
      # print('H1: ',H1)
      # print('right split :')
      H2 = calculate_entropy(data_after_split_right)
      p_H1 = len(data_after_split_left)/len(data_before_split)
      # print('H2: ',H2)
      p_H2 = len(data_after_split_right)/len(data_before_split)
      H_after_split = (p_H1*H1) + (p_H2*H2)
      information_gain = H_initial - H_after_split
      # print('H_initial - [(',len(data_after_split_left),'/',len(data_before_split),')* H1 + (',len(data_after_split_right),'/',len(data_before_split),')* H2]')
      # print('information_gain: ',information_gain)
      # print('*****************************************')
      if(information_gain > max_gain):
        max_gain = information_gain
        selected_threshold = threshold
        # print('feature :',k)
        # print('max_gain:',max_gain)
        # print('selected_threshold:',selected_threshold)
    if(max_gain > best_gain):
      best_gain = max_gain
      best_threshold = selected_threshold
      best_attribute = k
  # print('################################')
  # print('best_threshold:',best_threshold)
  # print('best_attribute:',best_attribute)
  # print('################################')
  return (best_attribute,best_threshold)

In [ ]:
def get_data_on_left(best_attribute,best_threshold,data_array):
  data_after_split_left = []
  for data in data_array:
    if(data[best_attribute] <= best_threshold):
      data_after_split_left.append(data)
  return data_after_split_left

def get_data_on_right(best_attribute,best_threshold,data_array):
  data_after_split_right = []
  for data in data_array:
    if(data[best_attribute] > best_threshold):
      data_after_split_right.append(data)
  return data_after_split_right
    



In [ ]:
def get_default_class(data_array):
  M_count = 0
  W_count = 0
  for data in data_array:
    if(data[3] == 'M'):
      M_count += 1
    else:
      W_count += 1
  if(M_count > W_count):
    return Node(None,None,None,'M')
  else:
    return Node(None,None,None,'W')

In [ ]:
class Node:

  def __init__(self,attribute,threshold,depth,class_label):
    self.left_node = None
    self.right_node = None
    self.max_depth = given_depth
    self.depth = depth
    self.attribute = attribute
    self.threshold = threshold
    self.class_label = class_label


  def print_tree(self):
    print(self.attribute,self.class_label)
    if self.left_node is not None: 
        self.left_node.print_tree()
    
    if self.right_node is not None:
        self.right_node.print_tree()

  def predict(self,data,max_depth):
    x1,x2,x3 = data
    temp = {0:x1,1:x2,2:x3}
    current_node = self
    i = 0
    while(i < max_depth):
      if(current_node.attribute in temp):
        if(temp[current_node.attribute] > current_node.threshold):
          if current_node.right_node is not None:
            current_node = current_node.right_node
        else:
          if current_node.left_node is not None: 
            current_node = current_node.left_node
        i += 1
      else:
        return current_node.class_label

    # print('######',current_node.class_label)
    return current_node.class_label

In [ ]:
def DTL(data_array,attributes,default,depth,max_depth,best_threshold):
  if(best_threshold == 0.0):
    return default
  if(len(data_array) == 0):
    return default
  elif(depth == max_depth):
    return default
  else:
    best_attribute,best_threshold = choose_attribute(data_array,attributes)
    if(best_threshold == 0.0):
      return default
    else:
      tree = Node(best_attribute,best_threshold,depth,None)
      left_data_set = get_data_on_left(best_attribute,best_threshold,data_array)
      right_data_set = get_data_on_right(best_attribute,best_threshold,data_array)
      tree.left_node = DTL(left_data_set,attributes,get_default_class(left_data_set),depth+1,max_depth,best_threshold)
      tree.right_node = DTL(right_data_set,attributes,get_default_class(right_data_set),depth+1,max_depth,best_threshold)

      return tree

In [ ]:

#read the training data
train_data = []
height_data = []
weight_data = []
age_data = []


with open('train.txt','r') as fileT:
  for line in fileT:
    result = line.strip().split(',')
    y = result[3].replace(')','').strip() #strip white spaces and split based on comma
    x1 = float(result[0].replace('(','').strip())
    x2 = float(result[1].strip())
    x3 = float(result[2].replace(')','').strip())
    train_data.append([x1,x2,x3,y]) #return as an array of features and class label

tree = DTL(train_data,3,get_default_class(train_data),0,given_depth,-1)
tree.print_tree()



1 None
0 None
2 None
None W
None M
0 None
None M
None W
0 None
0 None
None M
None M
None M


In [ ]:
# test the classifier

correct_count = 0
total_count = 0
test_data = []
with open('test.txt','r') as fileT:
  for line in fileT:
    result = line.strip().split(',')
    y = result[3].replace(')','').strip() #strip white spaces and split based on comma
    x1 = float(result[0].replace('(','').strip())
    x2 = float(result[1].strip())
    x3 = float(result[2].replace(')','').strip())
    test_data.append([x1,x2,x3,y])
    y_prediction = tree.predict((x1,x2,x3),given_depth)
    total_count += 1
    if(y == y_prediction):
      correct_count += 1
accuracy = correct_count/total_count
print('test data accuracy is :',accuracy*100)



test data accuracy is : 73.33333333333333


In [ ]:
correct_count_train = 0

for train in train_data:
  y_train_prediction = tree.predict((train[0],train[1],train[2]),given_depth)
  if(train[3] == y_train_prediction):
    correct_count_train += 1

train_accuracy = correct_count_train/len(train_data)
print('train data accuracy is :',train_accuracy*100)




train data accuracy is : 76.66666666666667
